# TABULAR PLAYGROND AGOSTO 2021

En esta edicion del concurso, se nos pide usar una regresión lineal para evaluar, sobre un conjunto de datos sinteticos, que suman 150.000 filas y 92 columnas 

Como viene siendo costumbre hasta ahora, lo primero que haremos será dividir el ejercicio en partes, primero,cargamos las librerias que vamos a usar y añadiendo las que necesitemos a medida que vamos avanzando en el trabajo, despues cargamos los datos y hacemos una exploracion en ellos, una vez obtenida la información que consideramos importante y sacado conclusiones en base a ello, se limpian los datos para que el entrenamiento nos resulte lo mejor posible. Con los dartos limpios, entrenamos los modelos y subimos los datos con el mejor de ellos

# **1- Cargamos las librerias**

pandas nos permite leer los datos del dataset

numpy es una libreria matematica

seaborn y matplotlib.pyplot  son librerias graficas

train_test_split nos permitirá dividir el dataset entre entrenamiento y validación

LinearRegression, XGBRegressor, ADA, CAT y LGBM crean modelos de regresion

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from catboost import CatBoostRegressor
from xgboost import XGBRegressor 
from lightgbm import LGBMRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn import metrics

# **2- Cargamos los datos**

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-aug-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-aug-2021/test.csv')

# 3- Mostramos los datos

In [ ]:
train.head(10)

In [ ]:
test.head(10)

**Tamño de los datos**

In [ ]:
train.shape, test.shape

**Descricion de los datos**

In [ ]:
train.describe()

In [ ]:
test.describe()

**Datos nulos**

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

Como hemos visto en esta pequeña exploracion, no hay datos nulos y todos los datos son de tipo numerico

In [ ]:
target = train["loss"].value_counts()
target

En la celda superior podemos ver la suma de los valores iguales en la columna objetivo, donde la mayor cuenta es al principio y va perdiendo cantidad a medida que avanza, haciendo el minimo en el ultimo valor de la columna 'loss'

In [ ]:
train.drop(columns=['id']).describe().T.style.bar(subset=['mean'])\
                            .background_gradient(subset=['std'])\
                            .background_gradient(subset=['min'])\
                            .background_gradient(subset=['50%'])

In [ ]:
train['f60'].describe

In [ ]:
test.drop(columns=['id']).describe().T.style.bar(subset=['mean'])\
                            .background_gradient(subset=['std'])\
                            .background_gradient(subset=['min'])\
                            .background_gradient(subset=['50%'])

In [ ]:
test['f60'].describe

In [ ]:
sns.distplot(train['loss'])

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
sns.countplot(data = train, 
              x = 'loss', 
              color = 'blue')

In [ ]:
fig, ax = plt.subplots(figsize = (40, 32))
sns.heatmap(train.corr(), annot = True, 
            fmt = ".2f", 
            cmap = 'coolwarm',
            cbar_kws = {"shrink": .8})

Vemos que no hay correlacion aparente entre los distintos valores de las columnas, por lo que no es necesario pensar en eliminar alguna de ellas

# 4- Modelos de regresion

**Separamos los datos en train y test**

In [ ]:
# Train data
X = train.drop(columns = ['loss'])
y = train['loss'].values
# Test data
X_test = test
print('Train set:', X.shape)
print('Test set:', X_test.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True)

**1- Regresion Lineal**

In [ ]:
model1_1 = LinearRegression()
model1_1.fit(X_train, y_train)
pred1_1 = model1_1.predict(X_test)

rmse1_1 = metrics.mean_squared_error(y_test, pred1_1, squared = False)
print('model 1_1 MSE score: ', rmse1_1)

**2- Cat Boost**

In [ ]:
model1_2 = CatBoostRegressor()
model1_2.fit(X_train, y_train, verbose = 0)
pred1_2 = model1_2.predict(X_test, verbose = 0)

rmse1_2 = metrics.mean_squared_error(y_test, pred1_2, squared = False)
print('model 1_2 MSE score: ', rmse1_2)

**3- Extreme Gradiente Boost Regresssor**

In [ ]:
model1_3 = XGBRegressor()
model1_3.fit(X_train, y_train, verbose = 0)
pred1_3 = model1_3.predict(X_test)

rmse1_3 = metrics.mean_squared_error(y_test, pred1_3, squared = False)
print('model 1_3 MSE score: ', rmse1_3)


**4- Light Gradient Regressor**

In [ ]:
model1_4 = LGBMRegressor()
model1_4.fit(X_train, y_train, verbose = 0)
pred1_4 = model1_4.predict(X_test)

rmse1_4 = metrics.mean_squared_error(y_test, pred1_4, squared = False)
print('model 4 MSE score: ', rmse1_4)

**5- Ada Boost Regressor**

In [ ]:
model1_5 = AdaBoostRegressor()
model1_5.fit(X_train, y_train)
pred1_5 = model1_5.predict(X_test)

rmse1_5 = metrics.mean_squared_error(y_test, pred1_5, squared = False)
print('model 1_5 MSE score: ', rmse1_5)

# 5- Subimos los datos

In [ ]:
sub = pd.read_csv('../input/tabular-playground-series-aug-2021/sample_submission.csv')
sub.head()

In [ ]:
test_pred_cat = model1_2.predict(test)
test_pred_lgbm = model1_4.predict(test)
test_pred = 0.6 * test_pred_cat + 0.4 * test_pred_lgbm
sub['loss'] = test_pred
sub.to_csv("submission.csv", index = False)